# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
import dice_ml
from dice_ml import Dice
from dice_ml.utils import helpers # helper functions


## Preliminaries: Loading the data and ML model

In [3]:
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
train_lbl, test_lbl = d.split_data(d.normalize_data(d.label_encoded_data, encoding='label'))
X_train_lbl = train_lbl.loc[:, train_lbl.columns != 'income']
y_train_lbl = train_lbl.loc[:, train_lbl.columns == 'income']
X_test_lbl = test_lbl.loc[:, test_lbl.columns != 'income']
y_test_lbl = test_lbl.loc[:, test_lbl.columns == 'income']
mlp_lbl = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=10, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp_lbl.fit(X_train_lbl, y_train_lbl.values.ravel())
m = dice_ml.Model(model=mlp_lbl, backend="sklearn")

C:\Users\gaugup\Anaconda3\envs\env1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Local feature importance

We first generate counterfactuals for a given input point. 

In [5]:
exp = Dice(d, m, method="genetic")
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}
e1 = exp.generate_counterfactuals(query_instance, total_CFs=50)
e1.visualize_as_dataframe()

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 09 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Other/Unknown,Assoc,Married,Other/Unknown,White,Female,45.0,0.0



Diverse Counterfactual set (new outcome:  1.0


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,79.0,Government,Bachelors,Divorced,Blue-Collar,Other,Male,80.0,1
1,30.0,Government,Assoc,Divorced,Blue-Collar,Other,Female,63.0,1
2,83.0,Other/Unknown,Assoc,Divorced,Other/Unknown,Other,Male,66.0,1
3,31.0,Government,Assoc,Divorced,Blue-Collar,Other,Female,53.0,1
4,59.0,Government,Bachelors,Divorced,Blue-Collar,White,Male,94.0,1
5,31.0,Government,Assoc,Divorced,Blue-Collar,Other,Female,84.0,1
6,57.0,Government,Bachelors,Divorced,Other/Unknown,White,Male,39.0,1
7,29.0,Government,Assoc,Divorced,Blue-Collar,White,Female,70.0,1
8,60.0,Government,Assoc,Divorced,Blue-Collar,White,Male,88.0,1
9,56.0,Other/Unknown,Assoc,Divorced,Blue-Collar,White,Male,66.0,1


These can now be used to calculate the feature importance scores. 

In [6]:
imp = exp.feature_importance([query_instance], cf_examples_list=[e1])
print(imp.local_importance)

[{'workclass': 0.64, 'education': 0.24, 'marital_status': 0.82, 'occupation': 0.66, 'race': 0.42, 'gender': 0.66, 'age': 0.96, 'hours_per_week': 1.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [7]:
imp = exp.feature_importance([query_instance])
print(imp.local_importance)

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
[{'workclass': 0.8, 'education': 0.2, 'marital_status': 1.0, 'occupation': 0.3, 'race': 0.5, 'gender': 0.8, 'age': 1.0, 'hours_per_week': 0.9}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [26]:
cobj=exp.feature_importance(dataset.iloc[0:10,:].to_dict('records'), total_CFs=10)
print(cobj.summary_importance)

Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 01 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactual

## Convert the counterfactual output to json

In [20]:
print(cobj.cf_examples_list)
print(cobj.local_importance)
print(cobj.summary_importance)
json_str = cobj.to_json()
print(json_str)

[<dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DADD1648>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DAA669C8>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DACB5348>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DACBE1C8>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DAA37C08>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DADD1148>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DAA2F048>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DAA3C588>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DACFB748>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x000001F7DAA3C148>]
[{'workclass': 0.4, 'education': 0.4, 'marital_status': 0.7, 'occupation': 0.8, 'race': 0.6, 'gender': 0.4, 'age': 1.0, 'hours_pe

## Convert the json output to a counterfactual object

In [24]:
imp_r=imp.from_json(json_str)
print(imp_r.cf_examples_list)
print(imp_r.local_importance)
print(imp_r.summary_importance)

[   age      workclass  education marital_status     occupation   race  gender  \
0   45  Other/Unknown      Assoc       Divorced    Blue-Collar  Other    Male   
1   69  Other/Unknown      Assoc       Divorced  Other/Unknown  Other    Male   
2   80     Government  Bachelors        Married    Blue-Collar  White  Female   
3   55     Government      Assoc        Married    Blue-Collar  White  Female   
4   71     Government  Bachelors       Divorced    Blue-Collar  White    Male   
5   40     Government      Assoc       Divorced    Blue-Collar  White    Male   
6   68  Other/Unknown      Assoc        Married    Blue-Collar  Other    Male   
7   63  Other/Unknown  Bachelors       Divorced    Blue-Collar  Other  Female   
8   45     Government      Assoc       Divorced  Other/Unknown  Other  Female   
9   66     Government  Bachelors       Divorced    Blue-Collar  Other    Male   

   hours_per_week  income  
0              74       1  
1              92       1  
2              50      